In [18]:
%matplotlib inline
import numpy as np
from pulp import *
import pandas as pd
import pprint
import csv
MEPS=1.03-6

In [19]:
df=pd.read_csv('premiere2.csv',header=0,usecols=[0])
df

,Club
0,Liverpool
1,Manchester City
2,Chelsea
3,Leicester City
4,Manchester United
5,Wolverhampton Wanderers
6,Tottenham Hotspur
7,Sheffield United
8,Burnley
9,Arsenal


In [20]:
DMUs=df.values
print(DMUs)

[['Liverpool']
 ['Manchester City']
 ['Chelsea']
 ['Leicester City']
 ['Manchester United']
 ['Wolverhampton Wanderers']
 ['Tottenham Hotspur']
 ['Sheffield United']
 ['Burnley']
 ['Arsenal']]


In [21]:
dfx=pd.read_csv('premiere2.csv',header=0,index_col=0,usecols=[0,3,4,5])
dfx

,Manager Salary(€M),Expenditure(€M),Seasons in Premiere League
Club,,,
Liverpool,24.0,10.40,28
Manchester City,27.0,166.82,23
Chelsea,7.5,45.00,28
Leicester City,9.1,104.30,14
Manchester United,8.3,214.00,28
Wolverhampton Wanderers,8.0,121.80,6
Tottenham Hotspur,23.0,148.50,28
Sheffield United,3.0,69.00,4
Burnley,3.6,19.40,6


In [22]:
x=dfx

In [23]:
print(dfx)

                         Manager Salary(€M)  Expenditure(€M)  \
Club                                                           
Liverpool                              24.0            10.40   
Manchester City                        27.0           166.82   
Chelsea                                 7.5            45.00   
Leicester City                          9.1           104.30   
Manchester United                       8.3           214.00   
Wolverhampton Wanderers                 8.0           121.80   
Tottenham Hotspur                      23.0           148.50   
Sheffield United                        3.0            69.00   
Burnley                                 3.6            19.40   
Arsenal                                 6.5           160.40   

                         Seasons in Premiere League  
Club                                                 
Liverpool                                        28  
Manchester City                                  23  
Chelsea        

In [24]:
x.shape

(10, 3)

In [26]:
dfy=pd.read_csv('premiere2.csv',header=0,usecols=[1,2])
dfy

,Profit,Points
0,152425146,93
1,150986355,75
2,146030216,63
3,123328078,62
4,142193180,62
5,127165114,59
6,145230801,58
7,96628865,54
8,107340558,54
9,142193180,53


In [27]:
y=dfy

In [28]:
y.shape

(10, 2)

In [29]:
n,m=x.shape
n,s=y.shape

In [30]:
res=[]

In [31]:
for o in range(n):
    prob=LpProblem('DMU_'+str(o),LpMaximize)
    #　重み
    v=[LpVariable('v'+str(i),lowBound=0,
                  cat='Continuous') for i in range(m)]
    u=[LpVariable('u'+str(i),lowBound=0,
                  cat='Continuous') for i in range(s)]
    #　目的関数
    prob += lpDot(u,y.iloc[o,:])

    # 制約条件
    prob += lpDot(v,x.iloc[o,:])==1,'Normalize'+str(o)

    for j in range(n):
        prob += lpDot(u,y.iloc[j,:]) <= lpDot(v,x.iloc[j,:])

    prob.solve()
    vs=np.array([v[i].varValue for i in range(m)]) #v*
    us=np.array([u[i].varValue for i in range(s)]) #u*
    print('DMU_'+str(o),': ',DMUs[o])
    print('vs: ',vs)
    print('us: ',us)
    print(' ')
#     # 参照集合作成
#     (eo,)=np.where(np.abs(np.dot(vs,x)-np.dot(us,y))<= MEPS)

#     res.append([DMUs[o],value(prob.objective),
#                 set(eo),tuple(vs),tuple(us)])

    res.append([DMUs[o],value(prob.objective)])

DMU_0 :  ['Liverpool']
vs:  [0.         0.02133541 0.02778971]
us:  [0.         0.01075269]
 
DMU_1 :  ['Manchester City']
vs:  [0.         0.00098743 0.03631637]
us:  [2.2084327e-09 0.0000000e+00]
 
DMU_2 :  ['Chelsea']
vs:  [0.12969803 0.00060588 0.        ]
us:  [4.4593308e-09 0.0000000e+00]
 
DMU_3 :  ['Leicester City']
vs:  [0.         0.00161499 0.05939692]
us:  [3.6119828e-09 0.0000000e+00]
 
DMU_4 :  ['Manchester United']
vs:  [0.12048193 0.         0.        ]
us:  [3.7405571e-09 0.0000000e+00]
 
DMU_5 :  ['Wolverhampton Wanderers']
vs:  [0.         0.         0.16666667]
us:  [6.8992497e-09 0.0000000e+00]
 
DMU_6 :  ['Tottenham Hotspur']
vs:  [0.         0.00084868 0.03121325]
us:  [1.8981072e-09 0.0000000e+00]
 
DMU_7 :  ['Sheffield United']
vs:  [0.30099329 0.00140609 0.        ]
us:  [1.0348875e-08 0.0000000e+00]
 
DMU_8 :  ['Burnley']
vs:  [0.         0.00416543 0.15319845]
us:  [9.3161431e-09 0.0000000e+00]
 
DMU_9 :  ['Arsenal']
vs:  [0.15384615 0.         0.        ]
u

In [32]:
print(res)

[[array(['Liverpool'], dtype=object), 0.999999984], [array(['Manchester City'], dtype=object), 0.3334432036358085], [array(['Chelsea'], dtype=object), 0.6511970399394528], [array(['Leicester City'], dtype=object), 0.44545889649305836], [array(['Manchester United'], dtype=object), 0.531881709020578], [array(['Wolverhampton Wanderers'], dtype=object), 0.8773438746149658], [array(['Tottenham Hotspur'], dtype=object), 0.27566362903986724], [array(['Sheffield United'], dtype=object), 1.0000000452768751], [array(['Burnley'], dtype=object), 0.9999999987618498], [array(['Arsenal'], dtype=object), 0.679172016847448]]


In [33]:
df_f=pd.DataFrame(res,columns=['Team','Efficiency'])
df_s=df_f.sort_values('Efficiency',ascending=False)
df_s

,Team,Efficiency
7,[Sheffield United],1.000000
8,[Burnley],1.000000
0,[Liverpool],1.000000
5,[Wolverhampton Wanderers],0.877344
9,[Arsenal],0.679172
2,[Chelsea],0.651197
4,[Manchester United],0.531882
3,[Leicester City],0.445459
1,[Manchester City],0.333443
6,[Tottenham Hotspur],0.275664


In [82]:
# def DEA_CCR(x,y,DMUs):
#     n,m=x.shape
#     n,s=y.shape
    
#     res=[]
    
#     for o in range(n):
#         prob=LpProblem('DMU_'+str(o),LpMaximize)
#         #　重み
#         v=[LpVariable('v'+str(i),lowBound=0,
#                       cat='Continuous') for i in range(m)]
#         u=[LpVariable('u'+str(i),lowBound=0,
#                       cat='Continuous') for i in range(s)]
#         #　目的関数
#         prob += lpDot(u,y.iloc[:,o])

#         # 制約条件
#         prob += lpDot(v,x.iloc[:,o])==1,'Normalize'+str(o)

#         for j in range(n):
#             prob += lpDot(u,y.iloc[:,j]) <= lpDot(v,x.iloc[:,j])

#         prob.solve()
#         vs=np.array([v[i].varValue for i in range(m)]) #v*
#         us=np.array([v[i].varValue for i in range(m)]) #u*
#         # 参照集合作成
#         (eo,)=np.where(np.abs(np.dot(vs,x)-np.dot(us,y))<= MEPS)
#         res.append([DMUs[o],value(prob.objective),
#                     set(eo),tuple(vs),tuple(us)])
#     return res